In [1]:
import re

data = []
with open('23.txt', 'r') as f:
    for line in f:
        caps = re.match(r'pos=<(-?\d+),(-?\d+),(-?\d+)>, r=(\d+)', line).groups()
        data.append([int(x) for x in caps])

# the desired point must be within the extrema for each dimension
minx, maxx = min(d[0] for d in data), max(d[0] for d in data)
miny, maxy = min(d[1] for d in data), max(d[1] for d in data)
minz, maxz = min(d[2] for d in data), max(d[2] for d in data)


In [16]:
import z3

def Abs(v):
    return z3.If(v >= 0, v, -v)

def Distance(p1, p2):
    x1, y1, z1 = p1
    x2, y2, z2 = p2
    return Abs(x2 - x1) + Abs(y2 - y1) + Abs(z2 - z1)

def BoolToInt(b):
    return z3.If(b, 1, 0)

o = z3.Optimize()

# optimizer variables
x = z3.Int('x')
y = z3.Int('y')
z = z3.Int('z')

# optimizer constraints
o.add(x >= minx, x <= maxx)
o.add(y >= miny, y <= maxy)
o.add(z >= minz, z <= maxz)

# optimizer goal
# find the count of points whose radius is within the target point
o.maximize(sum(BoolToInt(Distance((x, y, z), (a, b, c)) <= r)
               for a, b, c, r in data))

# run the optimizer and compute the solution distance from the origin
o.check()
m = o.model()
sum(m[i].as_long() for i in m)


164960498